In [1]:
import os
import numpy as np
import pandas as pd
from datetime import datetime, timedelta,date

# 
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(style="whitegrid")
sns.set_palette('icefire_r', 2)
import warnings

%matplotlib inline
#Przetwarzanie

# Baza danych gównych do nauki modelu
train = pd.read_csv("train.csv") 
# Baza danych Titanica do sprawdzenia modelu
test = pd.read_csv("test.csv")

In [2]:
train.head()

,id,date,country,store,product,num_sold
0,0,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Improve Your Coding,63
1,1,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Train More LLMs,66
2,2,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Win Friends and Influence People,9
3,3,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Win More Kaggle Competitions,59
4,4,2017-01-01,Argentina,Kaggle Learn,Using LLMs to Write Better,49


In [3]:
test.head()

,id,date,country,store,product
0,136950,2022-01-01,Argentina,Kaggle Learn,Using LLMs to Improve Your Coding
1,136951,2022-01-01,Argentina,Kaggle Learn,Using LLMs to Train More LLMs
2,136952,2022-01-01,Argentina,Kaggle Learn,Using LLMs to Win Friends and Influence People
3,136953,2022-01-01,Argentina,Kaggle Learn,Using LLMs to Win More Kaggle Competitions
4,136954,2022-01-01,Argentina,Kaggle Learn,Using LLMs to Write Better


In [4]:
print(pd.isnull(train).sum())

id          0
date        0
country     0
store       0
product     0
num_sold    0
dtype: int64


In [5]:
print(pd.isnull(test).sum())

id         0
date       0
country    0
store      0
product    0
dtype: int64


In [6]:
cols= ['country', 'store', 'product' ]
for col in cols:
  df_grouped = train.groupby(col)[col].count()
  df_grouped = df_grouped / train[col].count() * 100
  print(f"Unique value counts and percentages for column {col}:")
  print(df_grouped)

Unique value counts and percentages for column country:
country
Argentina    20.0
Canada       20.0
Estonia      20.0
Japan        20.0
Spain        20.0
Name: country, dtype: float64
Unique value counts and percentages for column store:
store
Kagglazon       33.333333
Kaggle Learn    33.333333
Kaggle Store    33.333333
Name: store, dtype: float64
Unique value counts and percentages for column product:
product
Using LLMs to Improve Your Coding                 20.0
Using LLMs to Train More LLMs                     20.0
Using LLMs to Win Friends and Influence People    20.0
Using LLMs to Win More Kaggle Competitions        20.0
Using LLMs to Write Better                        20.0
Name: product, dtype: float64


In [7]:
cols= ['country', 'store', 'product' ]
for col in cols:
  df_grouped = test.groupby(col)[col].count()
  df_grouped = df_grouped / test[col].count() * 100
  print(f"Unique value counts and percentages for column {col}:")
  print(df_grouped)

Unique value counts and percentages for column country:
country
Argentina    20.0
Canada       20.0
Estonia      20.0
Japan        20.0
Spain        20.0
Name: country, dtype: float64
Unique value counts and percentages for column store:
store
Kagglazon       33.333333
Kaggle Learn    33.333333
Kaggle Store    33.333333
Name: store, dtype: float64
Unique value counts and percentages for column product:
product
Using LLMs to Improve Your Coding                 20.0
Using LLMs to Train More LLMs                     20.0
Using LLMs to Win Friends and Influence People    20.0
Using LLMs to Win More Kaggle Competitions        20.0
Using LLMs to Write Better                        20.0
Name: product, dtype: float64


In [8]:
train['date'] = pd.to_datetime(train['date'])
test['date'] = pd.to_datetime(test['date'])

In [9]:
from sklearn.preprocessing import OrdinalEncoder
enc = OrdinalEncoder()
Enc_col = ['country', 'store', 'product' ]

# Fit the encoder and create the mapping for each column separately
categories = {}
for col in Enc_col:
    for df in [train, test]:
        df[col + '_Enc'] = enc.fit_transform(df[col].values.reshape(-1, 1))
    categories[col] = enc.categories_[0]  # Store categories for the current column

# Print the mapping
for col in Enc_col:
    print(f"Kolumna: {col}")
    for i, category in enumerate(categories[col]):
        print(f"  Wartość zakodowana: {i} -> Wartość początkowa: {category}")

Kolumna: country
  Wartość zakodowana: 0 -> Wartość początkowa: Argentina
  Wartość zakodowana: 1 -> Wartość początkowa: Canada
  Wartość zakodowana: 2 -> Wartość początkowa: Estonia
  Wartość zakodowana: 3 -> Wartość początkowa: Japan
  Wartość zakodowana: 4 -> Wartość początkowa: Spain
Kolumna: store
  Wartość zakodowana: 0 -> Wartość początkowa: Kagglazon
  Wartość zakodowana: 1 -> Wartość początkowa: Kaggle Learn
  Wartość zakodowana: 2 -> Wartość początkowa: Kaggle Store
Kolumna: product
  Wartość zakodowana: 0 -> Wartość początkowa: Using LLMs to Improve Your Coding
  Wartość zakodowana: 1 -> Wartość początkowa: Using LLMs to Train More LLMs
  Wartość zakodowana: 2 -> Wartość początkowa: Using LLMs to Win Friends and Influence People
  Wartość zakodowana: 3 -> Wartość początkowa: Using LLMs to Win More Kaggle Competitions
  Wartość zakodowana: 4 -> Wartość początkowa: Using LLMs to Write Better


In [10]:
categorical_features = ['id','country', 'store', 'product' ]
for df in [train, test]:
     df.drop(columns=categorical_features, inplace=True)

In [11]:
train.tail()

,date,num_sold,country_Enc,store_Enc,product_Enc
136945,2021-12-31,700,4.0,0.0,0.0
136946,2021-12-31,752,4.0,0.0,1.0
136947,2021-12-31,111,4.0,0.0,2.0
136948,2021-12-31,641,4.0,0.0,3.0
136949,2021-12-31,539,4.0,0.0,4.0


In [12]:
test.head()

,date,country_Enc,store_Enc,product_Enc
0,2022-01-01,0.0,1.0,0.0
1,2022-01-01,0.0,1.0,1.0
2,2022-01-01,0.0,1.0,2.0
3,2022-01-01,0.0,1.0,3.0
4,2022-01-01,0.0,1.0,4.0


In [13]:
train.dtypes

date           datetime64[ns]
num_sold                int64
country_Enc           float64
store_Enc             float64
product_Enc           float64
dtype: object

In [14]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136950 entries, 0 to 136949
Data columns (total 5 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   date         136950 non-null  datetime64[ns]
 1   num_sold     136950 non-null  int64         
 2   country_Enc  136950 non-null  float64       
 3   store_Enc    136950 non-null  float64       
 4   product_Enc  136950 non-null  float64       
dtypes: datetime64[ns](1), float64(3), int64(1)
memory usage: 5.2 MB


In [15]:
train[['country_Enc', 'store_Enc', 'product_Enc']] = train[['country_Enc', 'store_Enc', 'product_Enc']].astype(np.int8)
test[['country_Enc', 'store_Enc', 'product_Enc']] = test[['country_Enc', 'store_Enc', 'product_Enc']].astype(np.int8)
train['num_sold'] = train['num_sold'].astype(np.int16)

In [16]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 136950 entries, 0 to 136949
Data columns (total 5 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   date         136950 non-null  datetime64[ns]
 1   num_sold     136950 non-null  int16         
 2   country_Enc  136950 non-null  int8          
 3   store_Enc    136950 non-null  int8          
 4   product_Enc  136950 non-null  int8          
dtypes: datetime64[ns](1), int16(1), int8(3)
memory usage: 1.7 MB


In [17]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27375 entries, 0 to 27374
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   date         27375 non-null  datetime64[ns]
 1   country_Enc  27375 non-null  int8          
 2   store_Enc    27375 non-null  int8          
 3   product_Enc  27375 non-null  int8          
dtypes: datetime64[ns](1), int8(3)
memory usage: 294.2 KB


In [18]:
print(train['date'].isna().any()) 

False


In [19]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

import optuna
from optuna.study import Study
from optuna.trial import Trial
import optuna.storages

# # Definicja modelu GRU
# class GRUNet(nn.Module):
#   def __init__(self, input_size, hidden_size, output_size):
#     super(GRUNet, self).__init__()
#     self.gru = nn.GRU(input_size, hidden_size, num_layers=1)
#     self.fc = nn.Linear(hidden_size, output_size)

#   def forward(self, x):
#     x, _ = self.gru(x)
#     x = x[-1]  # Wybiera ostatni stan ukryty
#     x = self.fc(x)
#     return x

# # Funkcja ładowania danych
# def load_data():
#   # Załaduj dane treningowe i testowe
#   train_data = train
#   test_data = test

#   # Konwersja typów danych
#   train_data['date'] = pd.to_datetime(train_data['date']).dt.timestamp().astype(float)
#   test_data['date'] = pd.to_datetime(test_data['date']).dt.timestamp().astype(float)

#   # Konwersja kolumn na tensory PyTorch
#   train_tensors = {
#       'date': torch.from_numpy(train_data['date'].to_numpy()).float(),
#       'num_sold': torch.from_numpy(train_data['num_sold'].to_numpy()).long(),
#       'country_enc': torch.from_numpy(train_data['country_Enc'].to_numpy()).long(),
#       'store_enc': torch.from_numpy(train_data['store_Enc'].to_numpy()).long(),
#       'product_enc': torch.from_numpy(train_data['product_Enc'].to_numpy()).long(),
#   }
#   test_tensors = {
#       'date': torch.from_numpy(test_data['date'].to_numpy()).float(),
#       'country_enc': torch.from_numpy(test_data['country_Enc'].to_numpy()).long(),
#       'store_enc': torch.from_numpy(test_data['store_Enc'].to_numpy()).long(),
#       'product_enc': torch.from_numpy(test_data['product_Enc'].to_numpy()).long(),
#   }

#   # Utwórz zbiory danych PyTorch
#   train_dataset = torch.utils.data.Dataset(train_tensors)
#   test_dataset = torch.utils.data.Dataset(test_tensors)

#   # Zdefiniuj ładowarki danych
#   train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
#   test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

#   return train_loader, test_loader

# # Funkcja treningu
# def train(model, optimizer, train_loader, criterion):
#   model.train()
#   for data, target in train_loader:
#     optimizer.zero_grad()
#     output = model(data)
#     loss = criterion(output, target)
#     loss.backward()
#     optimizer.step()

# # Funkcja walidacji
# def validate(model, val_loader, criterion):
#   model.eval()
#   running_loss = 0.0
#   for data, target in val_loader:
#     output = model(data)
#     loss = criterion(output, target)
#     running_loss += loss.item()
#   val_loss = running_loss / len(val_loader)
#   return val_loss

# # Definicja funkcji obiektywnej dla Optuny
# def objective(trial: Trial):
#   # Pobierz hiperparametry z Optuny
#   input_size = trial.suggest_int('input_size', 10, 100)
#   hidden_size = trial.suggest_int('hidden_size', 32, 256)
#   output_size = trial.suggest_int('output_size', 1, 10)
#   learning_rate = trial.suggest_float('learning_rate', 1e-4, 1e-2)

#   # Utwórz model i kryterium
#   model = GRUNet(input_size, hidden_size, output_size)
#   criterion = nn.MSELoss()

#   # Zdefiniuj optymalizator
#   optimizer = optim.Adam(model.parameters(), lr=learning_rate)

#   # Pobierz dane
#   train_loader, val_loader = load_data()

#   # Trenuj i waliduj model
#   for epoch in range(10):
#     train(model, optimizer, train_loader, criterion)
#     val_loss = validate(model, val_loader, criterion)
#     trial.report(val_loss, step=epoch)

#   # Wybierz najlepszy model
#   if val_loss is not None:
#     return val_loss

# storage = optuna.storages.RDBStorage(
#     url="sqlite:///:memory:",
#     engine_kwargs={"pool_size": 20, "connect_args": {"timeout": 10}},
# )

# study = optuna.create_study(study_name="gru_model_optimization", storage=storage)
# study.optimize(objective, n_trials=100)

# # Wyświetl najlepsze znalezione hiperparametry
# best_trial = study.best_trial
# print('Najlepsze hiperparametry:')
# print('input_size:', best_trial.params['input_size'])
# print('hidden_size:', best_trial.params['hidden_size'])
# print('output_size:', best_trial.params['output_size'])
# print('learning_rate:', best_trial.params['learning_rate'])
# print('Najlepsza strata walidacyjna:', best_trial.value)

In [44]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import optuna
from optuna.study import Study
from optuna.trial import Trial
import optuna.storages

# Check data types
print(train['date'].dtype)
print(test['date'].dtype)

# Handle missing values (optional)
if train['date'].isna().any():
    print("There are missing values in 'date' column of train data")
    # Choose a handling method (e.g., remove rows, impute values)

if test['date'].isna().any():
    print("There are missing values in 'date' column of test data")
    # Choose a handling method for test data as well

train['year'] = pd.to_datetime(train['date']).dt.year
train['month'] = pd.to_datetime(train['date']).dt.month
train['day'] = pd.to_datetime(train['date']).dt.day

train_tensors = {
    'year': torch.from_numpy(train['year'].to_numpy()).long(),
    'month': torch.from_numpy(train['month'].to_numpy()).long(),
    'day': torch.from_numpy(train['day'].to_numpy()).long(),
    'num_sold': torch.from_numpy(train['num_sold'].to_numpy()).long(),
    'country_Enc': torch.from_numpy(train['country_Enc'].to_numpy()).long(),
    'store_Enc': torch.from_numpy(train['store_Enc'].to_numpy()).long(),
    'product_Enc': torch.from_numpy(train['product_Enc'].to_numpy()).long(),
}

test['year'] = pd.to_datetime(test['date']).dt.year
test['month'] = pd.to_datetime(test['date']).dt.month
test['day'] = pd.to_datetime(test['date']).dt.day

test_tensors = {
    'year': torch.from_numpy(test['year'].to_numpy()).long(),
    'month': torch.from_numpy(test['month'].to_numpy()).long(),
    'day': torch.from_numpy(test['day'].to_numpy()).long(),
    'country_Enc': torch.from_numpy(test['country_Enc'].to_numpy()).long(),
    'store_Enc': torch.from_numpy(test['store_Enc'].to_numpy()).long(),
    'product_Enc': torch.from_numpy(test['product_Enc'].to_numpy()).long(),
}

train_dataset = torch.utils.data.TensorDataset(
    train_tensors['year'],
    train_tensors['month'],
    train_tensors['day'],
    train_tensors['num_sold'],
    train_tensors['country_Enc'],
    train_tensors['store_Enc'],
    train_tensors['product_Enc'],
)

#### Szybka inspekcja
train_data_np = train_dataset.tensors

train_year = train_data_np[0].numpy()
train_month = train_data_np[1].numpy()
train_day = train_data_np[2].numpy()
train_num_sold = train_data_np[3].numpy()
train_country_enc = train_data_np[4].numpy()
train_store_enc = train_data_np[5].numpy()
train_product_enc = train_data_np[6].numpy()

train_df = pd.DataFrame({
    'year': train_year,
    'month': train_month,
    'day': train_day,
    'num_sold': train_num_sold,
    'country_Enc': train_country_enc,
    'store_Enc': train_store_enc,
    'product_Enc': train_product_enc
})

train_df.to_csv('prepared_train_data.csv', index=False)

####

val_dataset = torch.utils.data.TensorDataset(
    train_tensors['year'],
    train_tensors['month'],
    train_tensors['day'],
    train_tensors['country_Enc'],
    train_tensors['store_Enc'],
    train_tensors['product_Enc'],
)

  # Zdefiniuj ładowarki danych
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

datetime64[ns]
datetime64[ns]


In [62]:
# train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
# test_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

# Define GRU model
class GRUNet(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GRUNet, self).__init__()
        self.gru = nn.GRU(input_dim, hidden_dim, num_layers=1)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x, _ = self.gru(x)
        x = x[-1]  # Get the last hidden state
        x = self.fc(x)
        return x


def objective(trial):
    # Suggest hyperparameters with Optuna
    input_dim = trial.suggest_int('input_dim', 10, 100)
    hidden_dim = trial.suggest_int('hidden_size', 32, 256)  # Use hidden_size for consistency
    output_dim = trial.suggest_int('output_size', 1, 10)  # Use output_size for consistency
    learning_rate = trial.suggest_float('learning_rate', 1e-4, 1e-2)

    model = GRUNet(input_dim, hidden_dim, output_dim)
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters())

    # Implement training and validation functions
    def train(model, optimizer, train_loader, criterion):
       model.train()
       for data, target, extra_element in train_loader:  # Assuming an extra element
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()

    def validate(model, test_loader, criterion):
        model.eval()
        with torch.no_grad():
            losses = []
            for data, target in test_loader:
                output = model(data)
                loss = criterion(output, target)
                losses.append(loss.item())
            return sum(losses) / len(losses)

    # Training and Validation with Early Stopping (Optional)
    best_val_loss = float('inf')
    epochs_no_improvement = 0
    for epoch in range(25):
        train(model, optimizer, train_loader, criterion)
        val_loss = validate(model, test_loader, criterion)
        trial.report(val_loss, step=epoch)

        if val_loss < best_val_loss:
            best_val_loss = val_loss
            epochs_no_improvement = 0
        else:
            epochs_no_improvement += 1

        if epochs_no_improvement >= 5:  # Adjust this threshold as needed
            print("Early stopping at epoch", epoch)
            break

    return best_val_loss


# Create Optuna study and optimize
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)


[I 2024-05-08 20:24:27,643] A new study created in memory with name: no-name-aa37de2f-452d-4ef4-9b02-8d8485005620
[W 2024-05-08 20:24:27,651] Trial 0 failed with parameters: {'input_dim': 43, 'hidden_size': 200, 'output_size': 8, 'learning_rate': 0.009513325991333322} because of the following error: ValueError('too many values to unpack (expected 3)').
Traceback (most recent call last):
  File "C:\Users\domin\anaconda3\Lib\site-packages\optuna\study\_optimize.py", line 196, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "C:\Users\domin\AppData\Local\Temp\ipykernel_61800\1477774750.py", line 53, in objective
    train(model, optimizer, train_loader, criterion)
  File "C:\Users\domin\AppData\Local\Temp\ipykernel_61800\1477774750.py", line 32, in train
    for data, target, extra_element in train_loader:  # Assuming an extra element
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^
ValueError: too many values to unpack (expected 3)
[W 2024-05-08 20:24:27,652] T

ValueError: too many values to unpack (expected 3)